In [ ]:
import sys
# setting path
sys.path.append('..')
# importing
import ECC_library
from qiskit.circuit import Gate
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit import Aer, execute, IBMQ, BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.visualization import plot_bloch_multivector, plot_histogram

In [ ]:
def controlled_binary_doubling_cyclic(circuit,ctrl,x,n):
    for i in range(n-1,0,-1):
        circuit.ccx(ctrl,x[i-1],x[i])
        circuit.ccx(ctrl,x[i],x[i-1])
        circuit.ccx(ctrl,x[i-1],x[i])

In [ ]:
def test_controlled_doubling_modulo_p(a,p,n,g_s,ctrlq):
    x = QuantumRegister(n,name='x')
    c = QuantumRegister(1,name='d')
    g = QuantumRegister(1,name='g')
    ctrl = QuantumRegister(1,name='ctrl')
    clas_a = ClassicalRegister(n)
    clas_c = ClassicalRegister(1)
    clas_g = ClassicalRegister(1)
    circuit = QuantumCircuit(ctrl,x,c,g,clas_a,clas_c,clas_g)
    s_a=bin(int(a))[2:].zfill(n)
    s_p=bin(int(p))[2:].zfill(n)
    s_p=s_p[::-1]
    s_p1=s_p[::]+'0'
    if(g_s=='1'):
        circuit.x(g)
    if(ctrlq=='1'):
        circuit.x(ctrl)
    for i in range(0, n):
        if s_a[i] == '1':
            circuit.x(x[n-1-i])
    
    controlled_binary_doubling_cyclic(circuit,ctrl,x[0:n]+c[0:1],n+1)
    ECC_library.constant_subtraction(circuit,x[0:n]+c[0:1],g,s_p1,n+1)
    ECC_library.controlled_constant_addition(circuit,c,x,g,s_p,n)
    circuit.x(x[0])
    circuit.ccx(ctrl,x[0],c)
    circuit.x(x[0])
    circuit.x(ctrl)
    circuit.cx(ctrl,c)
    circuit.x(ctrl)
    for i in range(0, n):
        circuit.measure(x[i],clas_a[i])
    circuit.measure(c,clas_c)
    circuit.measure(g,clas_g)
    print('a:',a,'p:',p,'g:',g_s,'n:',n)
    backend = BasicAer.get_backend('qasm_simulator')
    simulation = execute(circuit, backend=backend ,shots=10)
    sim_result = simulation.result()
    out=list(sim_result.get_counts().keys())[0]
    output=out.split(' ')
    output_a = output[2]
    output_c = output[1]
    res_g = output[0]
    res_a = int(output_a, 2)
    res_c = int(output_c, 2)
    print('res_a:',res_a,'res_c:',res_c,'res_g:',res_g,'ctrl:',ctrlq)
    if ((res_a!=(a+a)%p and ctrlq=='1') or (res_a!=a and ctrlq=='0') or res_c!=0 or res_g!=g_s):
        print('found!')

In [ ]:
for n in range (4,6):
    for p in range (7,2**n,2):
        for i in range (0,p):
                test_controlled_doubling_modulo_p(i,p,n,'0','0')
                test_controlled_doubling_modulo_p(i,p,n,'1','0')
                test_controlled_doubling_modulo_p(i,p,n,'0','1')
                test_controlled_doubling_modulo_p(i,p,n,'1','1')
                print('-----------')